# 1.0 Case - Alavancagem de vendas

### 1.1 Bilibotecas

In [ ]:
# geral e visualizações
import warnings 
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from pandas_profiling import ProfileReport
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from plotly.subplots import make_subplots
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

# Cluster 
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

# Setar seed para obter resultados reproduziveis
seed = 100

# Classificação
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.model_selection import cross_val_score 
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

# feature selection
from sklearn.feature_selection import RFE

# Regressão
from sklearn.ensemble import RandomForestRegressor 
from sklearn.metrics import (mean_absolute_error,mean_squared_error,mean_absolute_percentage_error,r2_score)

### 1.2 Importando a base

In [ ]:
#importar base 
base = pd.read_csv('base.csv',sep = ';', decimal = ',' , encoding = 'latin-1')

In [ ]:
#resumo
print(base.shape)
base.head()

In [ ]:
#resumo
base.describe()

In [ ]:
#resumo
base.info()

In [ ]:
# alterando os tipos das variáveis
base['COD_CICLO'] = base['COD_CICLO'].astype('str')
base['COD_MATERIAL'] = base['COD_MATERIAL'].astype('str')

In [ ]:
base.info()

In [ ]:
# Utilizaremos apenas o período de 2018 a 2020, esse perído facilitará as análises
# Além disso, a qtd de informações em 2021 é pequena pois contém apenas um período
base = base[base['COD_CICLO']!= '202101']

### 1.3 Análise de contexto - Conhecendo as variáveis 

In [ ]:
# Verifcando os valores únicos
base.nunique()

In [ ]:
# Verifcando os valores em branco
base.isna().sum()

In [ ]:
# análise dos dados com profile
profile_base = ProfileReport(base)
profile_base

In [ ]:
# Quantidade de produtos por faixa de frequência de ciclos
freq_mat = base.groupby(['COD_MATERIAL'],as_index=False)['COD_CICLO'].nunique()
freq_mat = freq_mat.set_axis(['COD_MATERIAL', 'QTD_COD_CICLO'], axis=1)
freq_mat = pd.DataFrame(freq_mat['QTD_COD_CICLO'])
freq_mat.reset_index(inplace = True,drop = True)
freq_mat

print(freq_mat.min())
print(freq_mat.max())

plt.figure()
freq_mat.groupby(pd.cut(freq_mat['QTD_COD_CICLO'], np.arange(0,65,10))).count().plot(kind='bar', legend=False)
plt.title('Quantidade de materiais por faixa de frequência de ciclos')
plt.xlabel('')
plt.show()

In [ ]:
# Quantidade de produtos por faixa de ticket médio de venda líquida
ticket_mat = base.groupby(['COD_MATERIAL'],as_index=False).agg({
    'VL_RECEITA_LIQUIDA':'sum',
    'QT_VENDA_BRUTO':'sum'})
ticket_mat = ticket_mat.set_axis(['COD_MATERIAL', 'VL_RECEITA_LIQUIDA','QT_VENDA_BRUTA'], axis=1)
ticket_mat['TICKET_VENDA'] = ticket_mat['VL_RECEITA_LIQUIDA']/ticket_mat['QT_VENDA_BRUTA']
ticket_mat = pd.DataFrame(ticket_mat['TICKET_VENDA'])
ticket_mat.reset_index(inplace = True,drop = True)
ticket_mat

print(ticket_mat.min())
print(ticket_mat.max())

plt.figure()
ticket_mat.groupby(pd.cut(ticket_mat['TICKET_VENDA'], np.arange(0,250,60))).count().plot(kind='bar', legend=False)
plt.title('Quantidade de materiais por faixa de ticket médio de receita líquida')
plt.xlabel('')
plt.show()

In [ ]:
# Quantidade de produtos por faixa de média de preço
preco_mat = base.groupby(['COD_MATERIAL'],as_index=False)['VL_PRECO'].mean()
preco_mat = preco_mat.set_axis(['COD_MATERIAL', 'VL_PRECO'], axis=1)
preco_mat = pd.DataFrame(preco_mat['VL_PRECO'])
preco_mat.reset_index(inplace = True,drop = True)
preco_mat

print(preco_mat.min())
print(preco_mat.max())

plt.figure()
preco_mat.groupby(pd.cut(preco_mat['VL_PRECO'], np.arange(0,2500,500))).count().plot(kind='bar', legend=False)
plt.title('Quantidade de materiais por faixa de média de preço')
plt.xlabel('')
#plt.show()

In [ ]:
#VL_RECEITA_LIQUIDA por Canal
venda_liq_canal = base.groupby('COD_CANAL', as_index=False)['VL_RECEITA_LIQUIDA'].sum()
venda_liq_canal = venda_liq_canal.set_axis(['COD_CANAL', 'VL_RECEITA_LIQUIDA'], axis=1, inplace=False)
venda_liq_canal = venda_liq_canal.sort_values(by=['VL_RECEITA_LIQUIDA'],ascending=False)

plt.bar(venda_liq_canal['COD_CANAL'] ,venda_liq_canal['VL_RECEITA_LIQUIDA'])

In [ ]:
#VL_RECEITA_LIQUIDA por Categoria
venda_liq_categoria = base.groupby('DES_CATEGORIA_MATERIAL', as_index=False)['VL_RECEITA_LIQUIDA'].sum()
venda_liq_categoria = venda_liq_categoria.set_axis(['DES_CATEGORIA_MATERIAL', 'VL_RECEITA_LIQUIDA'], axis=1, inplace=False)
venda_liq_categoria = venda_liq_categoria.sort_values(by=['VL_RECEITA_LIQUIDA'],ascending=False)

plt.bar(venda_liq_categoria['DES_CATEGORIA_MATERIAL'] ,venda_liq_categoria['VL_RECEITA_LIQUIDA'])

In [ ]:
#VL_RECEITA_LIQUIDA por marca
venda_liq_marca = base.groupby('DES_MARCA_MATERIAL', as_index=False)['VL_RECEITA_LIQUIDA'].sum()
venda_liq_marca = venda_liq_marca.set_axis(['DES_MARCA_MATERIAL', 'VL_RECEITA_LIQUIDA'], axis=1, inplace=False)
venda_liq_marca = venda_liq_marca.sort_values(by=['VL_RECEITA_LIQUIDA'],ascending=False)

plt.bar(venda_liq_marca['DES_MARCA_MATERIAL'] ,venda_liq_marca['VL_RECEITA_LIQUIDA'])

In [ ]:
#VL_RECEITA_LIQUIDA por região
venda_liq_regiao = base.groupby('COD_REGIAO', as_index=False)['VL_RECEITA_LIQUIDA'].sum()
venda_liq_regiao = venda_liq_regiao.set_axis(['COD_REGIAO', 'VL_RECEITA_LIQUIDA'], axis=1, inplace=False)
venda_liq_regiao = venda_liq_regiao.sort_values(by=['VL_RECEITA_LIQUIDA'],ascending=False)

plt.bar(venda_liq_regiao['COD_REGIAO'] ,venda_liq_regiao['VL_RECEITA_LIQUIDA'])

In [ ]:
#QTD_VENDA por canal
qtd_canal = base.groupby('COD_CANAL', as_index=False)['QT_VENDA_BRUTO'].sum()
qtd_canal = qtd_canal.set_axis(['COD_CANAL', 'QT_VENDA_BRUTO'], axis=1, inplace=False)
qtd_canal = qtd_canal.sort_values(by=['QT_VENDA_BRUTO'],ascending=False)

plt.bar(qtd_canal['COD_CANAL'] ,qtd_canal['QT_VENDA_BRUTO'])

In [ ]:
#QTD_VENDA por categoria
qtd_categoria = base.groupby('DES_CATEGORIA_MATERIAL', as_index=False)['QT_VENDA_BRUTO'].sum()
qtd_categoria = qtd_categoria.set_axis(['DES_CATEGORIA_MATERIAL', 'QT_VENDA_BRUTO'], axis=1, inplace=False)
qtd_categoria = qtd_categoria.sort_values(by=['QT_VENDA_BRUTO'],ascending=False)

plt.bar(qtd_categoria['DES_CATEGORIA_MATERIAL'] ,qtd_categoria['QT_VENDA_BRUTO'])

In [ ]:
#QTD_VENDA por marca
qtd_marca = base.groupby('DES_MARCA_MATERIAL', as_index=False)['QT_VENDA_BRUTO'].sum()
qtd_marca = qtd_marca.set_axis(['DES_MARCA_MATERIAL', 'QT_VENDA_BRUTO'], axis=1, inplace=False)
qtd_marca = qtd_marca.sort_values(by=['QT_VENDA_BRUTO'],ascending=False)

plt.bar(qtd_marca['DES_MARCA_MATERIAL'] ,qtd_marca['QT_VENDA_BRUTO'])

In [ ]:
#QTD_VENDA por região
qtd_regiao = base.groupby('COD_REGIAO', as_index=False)['QT_VENDA_BRUTO'].sum()
qtd_regiao = qtd_regiao.set_axis(['COD_REGIAO', 'QT_VENDA_BRUTO'], axis=1, inplace=False)
qtd_regiao = qtd_regiao.sort_values(by=['QT_VENDA_BRUTO'],ascending=False)

plt.bar(qtd_regiao['COD_REGIAO'] ,qtd_regiao['QT_VENDA_BRUTO'])

### 1.4 Clusterização

#### 1.4.1 Prepando a base de dados para o cluster

In [ ]:
df_cluster = base
df_cluster

In [ ]:
#função para extrair o ano
def extrair_ano(arg):
     return arg[:4]

In [ ]:
df_cluster['ANO'] = df_cluster.COD_CICLO.apply(extrair_ano)

In [ ]:
df_cluster['ANO'].unique()

In [ ]:
df_cluster

In [ ]:
df_cluster = pd.pivot_table(df_cluster, values = ['QT_VENDA_BRUTO','QT_DEVOLUCAO','VL_RECEITA_BRUTA','VL_RECEITA_LIQUIDA'],index = ['COD_MATERIAL'],
            columns = ['ANO'], aggfunc = np.sum, fill_value=0)
    
df_cluster.reset_index(inplace=True)   

In [ ]:
df_cluster

In [ ]:
#renomear
df_cluster = df_cluster.set_axis(['COD_MATERIAL','QT_DEVOLUCAO_2018','QT_DEVOLUCAO_2019','QT_DEVOLUCAO_2020',
                                 'QT_VENDA_BRUTO_2018','QT_VENDA_BRUTO_2019','QT_VENDA_BRUTO_2020',
                                 'VL_RECEITA_BRUTA_2018','VL_RECEITA_BRUTA_2019','VL_RECEITA_BRUTA_2020',
                                 'VL_RECEITA_LIQUIDA_2018','VL_RECEITA_LIQUIDA_2019','VL_RECEITA_LIQUIDA_2020'], axis=1, inplace = False)

In [ ]:
# colunas
df_cluster['QT_DEVOLUCAO'] = df_cluster['QT_DEVOLUCAO_2018'] + df_cluster['QT_DEVOLUCAO_2019'] + df_cluster['QT_DEVOLUCAO_2020']
df_cluster['QT_VENDA_BRUTO'] = df_cluster['QT_VENDA_BRUTO_2018'] + df_cluster['QT_VENDA_BRUTO_2019'] + df_cluster['QT_VENDA_BRUTO_2020']
df_cluster['VL_RECEITA_BRUTA'] = df_cluster['VL_RECEITA_BRUTA_2018'] + df_cluster['VL_RECEITA_BRUTA_2019'] + df_cluster['VL_RECEITA_BRUTA_2020']
df_cluster['VL_RECEITA_LIQUIDA'] = df_cluster['VL_RECEITA_LIQUIDA_2018'] + df_cluster['VL_RECEITA_LIQUIDA_2019'] + df_cluster['VL_RECEITA_LIQUIDA_2020']

In [ ]:
df_cluster['VAR_RECEITA'] = (df_cluster['VL_RECEITA_LIQUIDA'] - df_cluster['VL_RECEITA_BRUTA']) / df_cluster['VL_RECEITA_BRUTA']
df_cluster['PROP_DEVOLUCAO'] = df_cluster['QT_DEVOLUCAO']/df_cluster['QT_VENDA_BRUTO']

In [ ]:
df_cluster = df_cluster.drop(['QT_DEVOLUCAO_2018','QT_DEVOLUCAO_2019','QT_DEVOLUCAO_2020',
                              'QT_VENDA_BRUTO_2018','QT_VENDA_BRUTO_2019','QT_VENDA_BRUTO_2020',
                              'VL_RECEITA_BRUTA_2018','VL_RECEITA_BRUTA_2019','VL_RECEITA_BRUTA_2020',
                              'QT_DEVOLUCAO','VL_RECEITA_BRUTA'],axis=1)

In [ ]:
# 2019/2018
df_cluster['VAR_1'] = (df_cluster['VL_RECEITA_LIQUIDA_2019'] -  df_cluster['VL_RECEITA_LIQUIDA_2018'])/df_cluster['VL_RECEITA_LIQUIDA_2018']

# 2020/2019
df_cluster['VAR_2'] = (df_cluster['VL_RECEITA_LIQUIDA_2020'] -  df_cluster['VL_RECEITA_LIQUIDA_2019'])/df_cluster['VL_RECEITA_LIQUIDA_2019']

df_cluster.replace([np.inf,-np.inf],1,inplace=True)
df_cluster = df_cluster.fillna(0)

#média de variações
df_cluster['MEDIA_VAR'] = (df_cluster['VAR_1'] + df_cluster['VAR_2'])/2

In [ ]:
#preço
df_preco = base.groupby(['COD_MATERIAL'], as_index=False)['VL_PRECO'].mean()
df_preco  = df_preco.set_axis(['COD_MATERIAL','MEDIA_PRECO'], axis=1, inplace=False)
df_preco  = df_preco.sort_values(by=['MEDIA_PRECO'],ascending=False)
df_preco

In [ ]:
df_cluster = pd.merge(df_cluster,df_preco,on= ['COD_MATERIAL'], how='left' )
df_cluster 

In [ ]:
df_cluster

In [ ]:
#quantidade de venda por flag de evento
df_qtd = base.groupby(['COD_MATERIAL','FLG_DATA'], as_index=False)['QT_VENDA_BRUTO'].sum()
df_qtd  = df_qtd.set_axis(['COD_MATERIAL','FLG_DATA','QT_VENDA_BRUTO'], axis=1, inplace=False)
df_qtd  = df_qtd.sort_values(by=['QT_VENDA_BRUTO'],ascending=False)

df_qtd = pd.pivot_table(df_qtd , values = ['QT_VENDA_BRUTO'],index = ['COD_MATERIAL'],
            columns = ['FLG_DATA'], aggfunc = np.sum, fill_value=0)
    
df_qtd.reset_index(inplace=True)  
df_qtd = df_qtd.set_axis(['COD_MATERIAL','QT_VENDA_BRUTO_0','QT_VENDA_BRUTO_1'], axis=1, inplace=False)

In [ ]:
df_cluster = pd.merge(df_cluster,df_qtd,on= ['COD_MATERIAL'], how='left' )
df_cluster 

In [ ]:
#recencia
aux = ('202008', '202009', '202010','202011','202012','202013','202014','202015','202016','202017')
df_recencia = base[base.COD_CICLO.isin(aux)]
df_recencia

df_recencia_2 = df_recencia.groupby('COD_MATERIAL', as_index=False)['COD_CICLO'].nunique()

df_recencia_2 = df_recencia_2.set_axis(['COD_MATERIAL','RECENCIA'],axis=1, inplace= False)
df_recencia_2

print(df_recencia_2.RECENCIA.max())
print(df_recencia_2.RECENCIA.min())

In [ ]:
#frequencia
df_frequencia =  base[base['COD_CICLO'] != '202101']

df_frequencia_2 = df_frequencia.groupby('COD_MATERIAL', as_index=False)['COD_CICLO'].nunique()

df_frequencia_2 = df_frequencia_2.set_axis(['COD_MATERIAL','FREQUENCIA'],axis=1, inplace= False)

print(df_frequencia_2.FREQUENCIA.max())
print(df_frequencia_2.FREQUENCIA.min())

In [ ]:
#valor
df_valor =  base[base['COD_CICLO'] != '202101']

df_valor_2 = df_valor.groupby(['COD_MATERIAL']).agg(
{'VL_RECEITA_LIQUIDA': 'sum',
 'COD_CICLO': 'nunique'}
)

df_valor_2['TICKET_CICLO'] = df_valor_2['VL_RECEITA_LIQUIDA']/df_valor_2['COD_CICLO']
df_valor_2 = df_valor_2.drop(['COD_CICLO','VL_RECEITA_LIQUIDA'],axis = 1)

In [ ]:
df_cluster = pd.merge(df_cluster,df_recencia_2,on= ['COD_MATERIAL'], how='left' )
df_cluster 

In [ ]:
df_cluster = pd.merge(df_cluster,df_frequencia_2,on= ['COD_MATERIAL'], how='left' )
df_cluster 

In [ ]:
df_cluster = pd.merge(df_cluster,df_valor_2,on= ['COD_MATERIAL'], how='left' )
df_cluster 

In [ ]:
df_cluster = df_cluster.drop(['VL_RECEITA_LIQUIDA_2018','VL_RECEITA_LIQUIDA_2019','VL_RECEITA_LIQUIDA_2020','VAR_1','VAR_2'],axis = 1)

In [ ]:
df_cluster.info()

In [ ]:
df_cluster

In [ ]:
df_cluster = df_cluster.fillna(0)

In [ ]:
df_cluster.info()

#### 1.4.2 Padronizando o dataframe e gerando o modelo Kmeans

In [ ]:
# Padronizando as variáveis
escala=StandardScaler()
df_cluster_normal = pd.DataFrame(escala.fit_transform(df_cluster.iloc[:,1:12]),columns=['QT_VENDA_BRUTO','VL_RECEITA_LIQUIDA','VAR_RECEITA','PROP_DEVOLUCAO','MEDIA_VAR','MEDIA_PRECO','QT_VENDA_BRUTO_0','QT_VENDA_BRUTO_1','RECENCIA','FREQUENCIA','TICKET_CICLO'])

In [ ]:
df_cluster_normal

In [ ]:
# Cálculo das iterações do k-means (Cotovelo)
# O número de iterações é subjetivo, utilizou-se 10, pois com esse número é possível avaliar
# o ponto de flexão da curva, mas se desejar é possível testar com valor maior ou menor

wcss = {}
for k in range(1, 15):
  print(k)
  kmeans = KMeans(n_clusters=k, init='k-means++', max_iter=200, random_state=3)
  kmeans.fit(df_cluster_normal)
  wcss[k] = kmeans.inertia_

# No código abaixo será gerado o gráfico que permite aplicação do método do "cotovelo"
fig = make_subplots(
    rows=1,
    cols=1,
    subplot_titles=['Método do Cotovelo'],
    shared_xaxes=True,
    shared_yaxes=False)


fig.add_trace(
    go.Scatter(
        x=list(wcss.keys()),
        y=list(wcss.values()),
        mode='lines+markers',
        textposition='top center',
        showlegend=False),
    row=1, 
    col=1)

fig.update_xaxes(visible=True, title='N° de clusters',row=1, col=1)
fig.update_yaxes(visible=True, title='Soma dos quadrados totais dos clusters',row=1, col=1)

fig.update_layout(
  title='Definição do Nº Ideal de Clusters',
  showlegend=False,
  xaxis_showticklabels=True,
  height=400,
  width=800,
  xaxis = dict(
    tickmode = 'linear',
    tick0 = 0,
    dtick = 1))

fig.show()

In [ ]:
#silhouette_score

maior_score = 0
i_otimo = 2

for i in range(2, 15):
    clusterer = KMeans(n_clusters=i)
    preds = clusterer.fit_predict(df_cluster_normal)
    score = silhouette_score(df_cluster_normal, preds)

    if score > maior_score:
        maior_score = score
        i_otimo = i
    print('Silhouette score para ' + str(i) + ' clusters: ' +str(score))
    
print('Maior score: ' + str(maior_score))
print('Número ideal de clusters: ' + str(i_otimo))

In [ ]:
# Definição do DF a ser Clusterizado
# tamanho de 5 clusters
# Clusterização
clus = KMeans(n_clusters = i_otimo, init='k-means++', max_iter=300,random_state=3)
clus.fit(df_cluster_normal)

# Ajustando as colunas
df_cluster_normal.loc[:, 'CLUSTER'] = clus.labels_
df_cluster_normal.head()

In [ ]:
# Cluster
# Dataset não padronizado
df_cluster.loc[:, 'CLUSTER'] = clus.labels_
df_cluster.head()

#### 1.4.3 Distribuição do cluster

In [ ]:
df_mat = df_cluster.groupby(['COD_MATERIAL','CLUSTER'],as_index=False)['VL_RECEITA_LIQUIDA'].sum()
df_mat = df_mat.set_axis(['COD_MATERIAL','CLUSTER','VL_RECEITA_LIQUIDA'], axis=1, inplace=False)
df_mat = df_mat.drop(['VL_RECEITA_LIQUIDA'],axis=1)
df_mat 

base2 = pd.merge(base,df_mat,on= ['COD_MATERIAL'], how='left')
base2

In [ ]:
# Volumetria  - quantidade de materiais por cluster
df_cluster['CLUSTER'].value_counts().plot(kind="bar")
plt.title('Qtd de produtos por Cluster')
plt.xlabel('Cluster')
plt.ylabel('Qtd de produtos')
plt.show()

In [ ]:
# Distribuição de VL_RECEITA_LIQUIDA por CLUSTER
plt.figure(figsize=(10, 5))

# plot a bar chart
sns.barplot(
    y="VL_RECEITA_LIQUIDA", 
    x="CLUSTER", 
    data=df_cluster, 
    estimator=sum, 
    ci=None );

plt.title('Valor de receita líquida por Cluster')
plt.xlabel('Cluster')
plt.ylabel('Valor de venda líquida')

In [ ]:
# Distribuição de VL_RECEITA_BRUTA por CLUSTER
plt.figure(figsize=(10, 5))

# plot a bar chart
sns.barplot(
    y="VL_RECEITA_BRUTA", 
    x="CLUSTER", 
    data=base2, 
    estimator=sum, 
    ci=None );

plt.title('Valor de receita bruta por Cluster')
plt.xlabel('Cluster')
plt.ylabel('Valor de venda bruta')

In [ ]:
# Distribuição de QT_VENDA_BRUTO por CLUSTER
plt.figure(figsize=(10, 5))

# plot a bar chart
sns.barplot(
    y="QT_VENDA_BRUTO", 
    x="CLUSTER", 
    data=df_cluster, 
    estimator=sum, 
    ci=None );

plt.title('Quantidade de Venda Bruta por Cluster')
plt.xlabel('Cluster')
plt.ylabel('Quantidade')

In [ ]:
# Distribuição de QT_VENDA_BRUTO por CLUSTER
plt.figure(figsize=(10, 5))

# plot a bar chart
sns.barplot(
    y="QT_DEVOLUCAO", 
    x="CLUSTER", 
    data=base2, 
    estimator=sum, 
    ci=None );

plt.title('Quantidade de devoluções por Cluster')
plt.xlabel('Cluster')
plt.ylabel('Quantidade')

In [ ]:
## Média de recência por CLUSTER
cluster_recencia = df_cluster.groupby('CLUSTER', as_index=False)['RECENCIA'].mean()
cluster_recencia = cluster_recencia .set_axis(['CLUSTER', 'MEDIA_RECENCIA'], axis=1, inplace=False)
cluster_recencia = cluster_recencia .sort_values(by=['MEDIA_RECENCIA'],ascending=False)

plt.bar(cluster_recencia ['CLUSTER'] ,cluster_recencia ['MEDIA_RECENCIA'])
plt.title('Média de recência por Cluster')
plt.xlabel('Cluster')
plt.ylabel('Média de recencia')

In [ ]:
### Média de frequência por CLUSTER
cluster_frequencia = df_cluster.groupby('CLUSTER', as_index=False)['FREQUENCIA'].mean()
cluster_frequencia = cluster_frequencia.set_axis(['CLUSTER', 'MEDIA_FREQUENCIA'], axis=1, inplace=False)
cluster_frequencia = cluster_frequencia.sort_values(by=['MEDIA_FREQUENCIA'],ascending=False)

plt.bar(cluster_frequencia ['CLUSTER'] ,cluster_frequencia['MEDIA_FREQUENCIA'])
plt.title('Média de frequência por Cluster')
plt.xlabel('Cluster')
plt.ylabel('Média de frequência')

In [ ]:
# Média de ticket_ciclo
cluster_media_ciclo = df_cluster.groupby('CLUSTER', as_index=False)['TICKET_CICLO'].mean()
cluster_media_ciclo = cluster_media_ciclo.set_axis(['CLUSTER', 'MEDIA_TICKET_CICLO'], axis=1, inplace=False)
cluster_media_ciclo = cluster_media_ciclo.sort_values(by=['MEDIA_TICKET_CICLO'],ascending=False)

plt.bar(cluster_media_ciclo['CLUSTER'] ,cluster_media_ciclo['MEDIA_TICKET_CICLO'])
plt.title('Média de ticket ciclo por Cluster')
plt.xlabel('Cluster')
plt.ylabel('Média de frequência')

In [ ]:
#correlação entre as variáveis do cluster
sns.heatmap(data = df_cluster.corr(),cmap='coolwarm') 

In [ ]:
#correlação entre quantidade de campanhas e valor de receita líquida - Cluster 3
corr_campanha_cluster3 = base2[base2['CLUSTER'] == 1].groupby(['COD_CICLO',],as_index=False).agg(
    {'FLG_CAMPANHA_MKT_A':'sum',
     'FLG_CAMPANHA_MKT_B':'sum',
     'FLG_CAMPANHA_MKT_C':'sum',
     'FLG_CAMPANHA_MKT_D':'sum',
     'VL_RECEITA_LIQUIDA':'sum',
    })

corr_campanha_cluster3

plt.title("Quantidade de campanhas e valor de receita líquida - Cluster 3")
sns.heatmap(corr_campanha_cluster3.corr(),cmap='coolwarm')

In [ ]:
#correlação entre quantidade de campanhas e vquantidade de venda bruta - Cluster 3
corr_campanha_cluster3_qtd = base2[base2['CLUSTER'] == 1].groupby(['COD_CICLO',],as_index=False).agg(
    {'FLG_CAMPANHA_MKT_A':'sum',
     'FLG_CAMPANHA_MKT_B':'sum',
     'FLG_CAMPANHA_MKT_C':'sum',
     'FLG_CAMPANHA_MKT_D':'sum',
     'QT_VENDA_BRUTO':'sum',
    })

corr_campanha_cluster3_qtd

plt.title("Quantidade de campanhas e quantidade de venda bruta - Cluster 3")
sns.heatmap(corr_campanha_cluster3_qtd.corr(),cmap='coolwarm')

In [ ]:
#exportar resultado
df_cluster.to_excel('df_cluster_novo.xlsx')
base2.to_excel('base2_novo.xlsx')

#### 1.4.4 Resultados dos cluster

#### Cluster 1 - Modelo campeão

Utilizei o modelo K-means para clusterizar os materiais em grupos com característica comuns, e através desse modelo, melhorar os resultado de vendas da empresa. Obtive como grupo mais relevante o cluster 1 pois representa 50,38% da receita líquida com apenas 58 materiais, o que representa a oportunidade de desenvolver uma estratégia de venda com baixo esforço de implementação e alto impacto financeiro.

O cluster 1 é grupo de materiais com valor agregado alto (Média de preço: 839,95 reais) e uma boa recência em ciclos (Média de recência: 9,81 de 10), frequência em ciclos (Média de frequência: 44 de 52) e ticket-médio por ciclo (Média de ticket por ciclo: ~ 23 Milhões de reais).

Para alavancar as vendas é necessário focar no canal anon_s7(31,14% da venda líquida total), na categoria anon_S12 (43,30% da venda líquida total), e região anon_S1 (28,60% da venda líquida total). Além disso, o cluster possui uma quantidade pequena de 19 marcas. 

Sobre as campanhas de venda, a parcela maior de participação (6,61% da quantidade de campanhas) corresponde as campanhas B (5,22%) e D (1,39%). Na matriz de correlação da quantidade de vendas e quantidade de campanhas por flag, o cluster 1 teve uma correlação positiva alta de 0.6 com a campanha B e uma correlação positiva fraca de 0.2 com a campanha D. Sendo B e D as campanhas que obtiveram maior retorno financeiro.

Estratégias de venda para o cluster 1:

- Desenvolver campanhas de marketing que enfatizem o valor dos materiais para o cliente, independente do preço do material. É um cluster com materiais de valor agregado alto que precisa ser alinhado a um público-alvo específico, os que não se preocupam com preço, mas esperam valor (qualidade, diferenciação e outros) no material. 

- Garantir uma boa experiência de compra. Investir em um ambiente acolhedor, atendimento atencioso, transparência sobre as características dos produtos e flexibilidade de opções.

- É necessário direcionar os produtos para canais e regiões específicas,fazer uma boa análise dos dados garante um bom escoamento do produto. 

- Como o sucesso desse cluster depende principalmente do match com o público-alvo, investir em estratégias de marketing inbound
para conseguir novos cliente. Segundo SIQUEIRA,"No Inbound Marketing, é o cliente que procura pela empresa e não o contrário.
Ou seja, são realizadas ações com o intuito de atrair o potencial cliente para seu blog ou site e, a partir dessa atração, é feito todo um trabalho de relacionamento com essa pessoa. Esse relacionamento é desempenhado por meio de conteúdo personalizado e autoral".

- Outra estratégia para obter novos clientes é a demonstração do produto, nesse momento o cliente conhecerá o produto e despertará o desejo de compra.

- Para produtos com valor alto é uma estratégia investir em um pós-venda com serviços acima da média, o que gera valor aos olhos do cliente, atende a expectativa de diferenciação e retém o público-alvo.

- Para complementar a estratégia de vendas foi desenvolvido um modelo de classificação para identificar os materiais do cluster 1 e 2 modelos de regressão para prever a quantidade de venda bruta e valor de receita líquida do cluster 1.


Comentários sobre o cluster 0

O cluster 0 representa do 49,62% do total de vendas líquidas e 64,83% da quantidade total de vendas, no entanto, esse cluster possui 2172 materiais únicos, essa quantidade quando comparada a quantidade de materiais do cluster 0 (53 materiais) evidencia o alto esforço para desenvolver uma estratégia de alavancagem de venda nesse cluster.

Fontes:

SIQUEIRA, André.2022.Tudo sobre Inbound Marketing. Disponível em:<https://resultadosdigitais.com.br/marketing/o-que-e-inbound-marketing/>. Acesso em: 05/12/2022.



### 1.5 Classificação

#### 1.5.1 Preparação dos dados - Modelo 1 

In [ ]:
df_cluster_classe = df_cluster[['COD_MATERIAL','CLUSTER']]

In [ ]:
df_class = pd.merge(base[base.COD_CICLO !='202101'] ,df_cluster_classe,on= ['COD_MATERIAL'], how='left' )

In [ ]:
df_class

In [ ]:
df_class.CLUSTER.unique()

In [ ]:
df_class.info()

In [ ]:
df_class = df_class.fillna(0)

In [ ]:
# Concatena as colunas criadas ao dataframe
df_class_encoding = pd.concat([df_class.drop("COD_CANAL", axis=1), pd.get_dummies(df_class.COD_CANAL,prefix='CANAL', prefix_sep='_')], axis=1)
df_class_encoding.head()

In [ ]:
# Concatena as colunas criadas ao dataframe
df_class_encoding = pd.concat([df_class_encoding.drop("DES_CATEGORIA_MATERIAL", axis=1), pd.get_dummies(df_class.DES_CATEGORIA_MATERIAL,prefix='CATEGORIA', prefix_sep='_')], axis=1)
df_class_encoding.head()

In [ ]:
# Concatena as colunas criadas ao dataframe
df_class_encoding = pd.concat([df_class_encoding.drop("DES_MARCA_MATERIAL", axis=1), pd.get_dummies(df_class.DES_MARCA_MATERIAL,prefix='MARCA', prefix_sep='_')], axis=1)
df_class_encoding.head()

In [ ]:
# Concatena as colunas criadas ao dataframe
df_class_encoding = pd.concat([df_class_encoding.drop("COD_REGIAO", axis=1), pd.get_dummies(df_class.COD_REGIAO,prefix='REGIAO', prefix_sep='_')], axis=1)
df_class_encoding.head()

In [ ]:
df_class_encoding.info()

In [ ]:
df_class_encoding

In [ ]:
df_class_encoding['PROP_DEVOLUCAO'] = df_class_encoding['QT_DEVOLUCAO']/df_class_encoding['QT_VENDA_BRUTO']
df_class_encoding['VAR_RECEITA'] = (df_class_encoding['VL_RECEITA_LIQUIDA'] - df_class_encoding['VL_RECEITA_BRUTA'])/df_class_encoding['VL_RECEITA_BRUTA']

In [ ]:
df_class_encoding = df_class_encoding.drop(['COD_CICLO','QT_DEVOLUCAO','VL_RECEITA_BRUTA','COD_MATERIAL'],axis=1)

In [ ]:
df_class_encoding

#### 1.5.2 Classificação - Modelo 1 

In [ ]:
df_class_encoding['CLUSTER'].value_counts()

In [ ]:
#Split
X, y = df_class_encoding.drop('CLUSTER',axis=1), df_class_encoding.CLUSTER

print("X", X.shape)
print("y", y.shape)

# Usar train_test_split para criar o conjunto de teste
X_training, X_test, y_training, y_test = train_test_split(X, y, random_state=seed, test_size=0.25, stratify=y)

# Como saída, temos a porção de treino e teste para as variáveis X e Y

print("Test set X", X_test.shape)
print("Test set y", y_test.shape)

In [ ]:
# Random Forest Classifier - MODELO 1

# Gerar um objeto árvore usando uma semente (seed) para resultados reproduzíveis
modelo_1 = RandomForestClassifier(criterion = 'entropy', random_state=seed)

# Fitar, ou treinar, o modelo usando o dataset de treino
modelo_1.fit(X_training, y_training)

# Prever todos os resultados no dataset de validação
y_pred_1 = modelo_1.predict(X_test)

# Calcular e printar métricas

# A função score da nossa árvore calcula a acurácia, tendo como entradas as variáveis X e y e, por trás, essa função gerará o y previsto e irá comparar com o y real (variável y_train, por exemplo)
print("Acurácia no conjunto de treino: {:.3f}".format(modelo_1.score(X_training, y_training)))
print("Acurácia no conjunto de teste: {:.3f}".format(modelo_1.score(X_test, y_test)))

# O classification report traz diversas métricas interessantes, en formato de tabela, de forma simples
print(classification_report(y_test, y_pred_1))

# A matriz de confusão mostra de forma fácil os dados
# Verdadeiros Positivos | Falsos Positivos
# Falsos Negativos | Verdadeiros Negativos
print(confusion_matrix(y_test, y_pred_1))
print("Área sob a curva ROC: {:.3f}".format(roc_auc_score(y_test, y_pred_1)))

# Plotar importância das features
# Avalia quão importante é cada variável para as decisões que a árvore fornece

feature_scores_1 = pd.Series(modelo_1.feature_importances_, index=X.columns).sort_values(ascending=False)
print(feature_scores_1)

# Creating a seaborn bar plot
sns.barplot(x=feature_scores_1, y=feature_scores_1.index)

# Add labels to the graph
plt.xlabel('Importância das variáveis')
plt.ylabel('Variáveis')

# Add title to the graph
plt.title("Importância das variáveis")

# Visualize the graph
plt.show()

#### 1.5.3 Preparação dos dados - Modelo 2 

In [ ]:
df_class_encoding_2 = df_class
df_class_encoding_2

In [ ]:
df_class_encoding_2['PROP_DEVOLUCAO'] = df_class_encoding_2['QT_DEVOLUCAO']/df_class_encoding_2['QT_VENDA_BRUTO']
df_class_encoding_2['VAR_RECEITA'] = (df_class_encoding_2['VL_RECEITA_LIQUIDA'] - df_class_encoding_2['VL_RECEITA_BRUTA'])/df_class_encoding_2['VL_RECEITA_BRUTA']

In [ ]:
df_class_encoding_2 = df_class_encoding_2.drop(['COD_CICLO','COD_MATERIAL','QT_DEVOLUCAO','VL_RECEITA_BRUTA','ANO'],axis=1)

In [ ]:
df_class_encoding_2.CLUSTER.value_counts()

In [ ]:
# Concatena as colunas criadas ao dataframe
df_class_encoding_2 = pd.concat([df_class_encoding_2.drop("COD_CANAL", axis=1), pd.get_dummies(df_class_encoding_2.COD_CANAL,prefix='CANAL', prefix_sep='_')], axis=1)
df_class_encoding_2.head()

In [ ]:
# Aplica o One Hot Encoding na coluna agrupada e concatena o resultado ao dataframe df_class_encoding_2, descartando a coluna original
df_class_encoding_2 = pd.concat([df_class_encoding_2.drop("COD_REGIAO", axis=1), pd.get_dummies(df_class_encoding_2.COD_REGIAO,prefix='REGIAO', prefix_sep='_')], axis=1)
df_class_encoding_2.head()

In [ ]:
# Calcula o valor médio de VL_RECEITA_LIQUIDA por categoria de DES_CATEGORIA_MATERIAL
df_class_encoding_2.VL_RECEITA_LIQUIDA.groupby(df_class_encoding_2['DES_CATEGORIA_MATERIAL']).mean()

In [ ]:
# Agrupamento das categorias de DES_CATEGORIA_MATERIAL
def get_periods_of_category(arg1):
    '''Retorna os periodos de sol a partir da categoria Sun'''
    if arg1 == 'anon_S11' or arg1 == 'anon_S18' or arg1 =='anon_S30' or arg1 =='anon_S5':
        return '1'
    elif arg1 == 'anon_S2':
        return '2'
    elif arg1 == 'anon_S12':
        return '3'
    else:
        return 'outro'


df_class_encoding_2['CATEGORIA_PERIODO'] = df_class_encoding_2.DES_CATEGORIA_MATERIAL.apply(get_periods_of_category)
df_class_encoding_2.head()

In [ ]:
# Aplica o One Hot Encoding na coluna agrupada e concatena o resultado ao dataframe [df_class_encoding_2, descartando a coluna original
df_class_encoding_2 = pd.concat([df_class_encoding_2.drop("CATEGORIA_PERIODO", axis=1), pd.get_dummies(df_class_encoding_2['CATEGORIA_PERIODO'],prefix='CATEGORIA', prefix_sep='_' )], axis=1)
df_class_encoding_2.head()

In [ ]:
# Calcula o valor médio de VL_RECEITA_LIQUIDA por categoria de DES_MARCA_MATERIAL
df_categoria_marca = pd.DataFrame(df_class_encoding_2.VL_RECEITA_LIQUIDA.groupby(df_class_encoding_2['DES_MARCA_MATERIAL']).mean())
df_categoria_marca.reset_index(inplace=True)  
df_categoria_marca = df_categoria_marca.set_axis(['DES_MARCA_MATERIAL','MEDIA_MARCA'],axis=1, inplace= False)
df_categoria_marca

In [ ]:
df_class_encoding_2 = pd.merge(df_class_encoding_2, df_categoria_marca, on= ['DES_MARCA_MATERIAL'], how='left')
df_class_encoding_2.head()

In [ ]:
min = df_categoria_marca.MEDIA_MARCA.min()
max = df_categoria_marca.MEDIA_MARCA.max()

amplitude = max - min
n= 5 
tamanho = (amplitude/ n)

In [ ]:
# Agrupamento das categorias de DES_CATEGORIA_MATERIAL
def obter_intervalo_marca(arg1):
    '''Retorna os periodos de sol a partir da categoria Sun'''
    min_local = min
    max_local = max
    amplitude_local = (max_local - min_local)
    n_local = 5
    tamanho_local = float( amplitude_local/ n_local)
    
    if arg1 >= 0 and arg1 < tamanho_local:
        return '1'
    elif arg1 >= tamanho_local and arg1 < tamanho_local*2:
        return '2'
    elif arg1 >= tamanho_local*2 and arg1 < tamanho_local*3:
        return '3'
    elif arg1 >= tamanho_local*3 and arg1 < tamanho_local*4:
        return '4'
    elif arg1 >= tamanho_local*5:
        return '5'
    else:
        return 'outro'


df_class_encoding_2['CATEGORIA_MARCA'] = df_class_encoding_2.MEDIA_MARCA.apply(obter_intervalo_marca)
df_class_encoding_2.head()

In [ ]:
df_class_encoding_2['CATEGORIA_MARCA'].unique()

In [ ]:
# Aplica o One Hot Encoding na coluna agrupada e concatena o resultado ao dataframe [df_class_encoding_2, descartando a coluna original
df_class_encoding_2 = pd.concat([df_class_encoding_2.drop("CATEGORIA_MARCA", axis=1), pd.get_dummies(df_class_encoding_2['CATEGORIA_MARCA'],prefix='MARCA', prefix_sep='_' )], axis=1)
df_class_encoding_2.head()

In [ ]:
df_class_encoding_2.info()

In [ ]:
df_class_encoding_2 = df_class_encoding_2.drop(['DES_CATEGORIA_MATERIAL','DES_MARCA_MATERIAL','MEDIA_MARCA'],axis =1)

In [ ]:
df_class_encoding_2

In [ ]:
df_class_encoding_2.info()

#### 1.5.4 Classificação - Modelo 2

In [ ]:
df_class_encoding_2.info()

In [ ]:
#matriz de correlação

plt.figure(figsize=(10,5))
sns.heatmap(df_class_encoding_2.corr(),cmap='coolwarm')

In [ ]:
df_class_encoding_2.CLUSTER.value_counts()

In [ ]:
df_class_encoding_2.info()

In [ ]:
#Split
X2, y2 = df_class_encoding_2.drop(['CLUSTER'],axis=1), df_class_encoding_2['CLUSTER']

print("X", X2.shape)
print("y", y2.shape)

# Usar train_test_split para criar o conjunto de teste
X2_training, X2_test, y2_training, y2_test = train_test_split(X2, y2, random_state=seed, test_size=0.25, stratify=y)

# Como saída, temos a porção de treino e teste para as variáveis X e Y
print("Test set X", X2_test.shape)
print("Test set y", y2_test.shape)

In [ ]:
# Random Forest Classifier - MODELO 2

# Gerar um objeto árvore usando uma semente (seed) para resultados reproduzíveis
modelo_2 = RandomForestClassifier(criterion = 'entropy',random_state=seed)

# Fitar, ou treinar, o modelo usando o dataset de treino
modelo_2.fit(X2_training, y2_training)

# Usa o modelo para prever o conjunto de teste
y_pred_2 = modelo_2.predict(X2_test)

In [ ]:
# Calcular e printar métricas

# A função score da nossa árvore calcula a acurácia, tendo como entradas as variáveis X e y e, por trás, essa função gerará o y previsto e irá comparar com o y real
print("Acurácia no conjunto de treino: {:.3f}".format(modelo_2.score(X2_training, y2_training)))
print("Acurácia no conjunto de teste: {:.3f}".format(modelo_2.score(X2_test, y2_test)))

# O classification report traz diversas métricas interessantes, en formato de tabela, de forma simples
print(classification_report(y2_test, y_pred_2))

# A matriz de confusão mostra de forma fácil os dados
# Verdadeiros Positivos | Falsos Positivos
# Falsos Negativos | Verdadeiros Negativos
print(confusion_matrix(y2_test, y_pred_2))
print("Área sob a curva ROC: {:.3f}".format(roc_auc_score(y2_test, y_pred_2)))

# Plotar importância das features
# Avalia quão importante é cada variável para as decisões que o modleo fornece

feature_scores_2 = pd.Series(modelo_2.feature_importances_, index=X2.columns).sort_values(ascending=False)
print(feature_scores_2)

# Criando um plot
sns.barplot(x=feature_scores_2, y=feature_scores_2.index)

# labels
plt.xlabel('Importância das variáveis')
plt.ylabel('Variáveis')

# título
plt.title("Importância das variáveis")

# visualizar
plt.show()

#### 1.5.5 Classificação - Modelo 3

In [ ]:
df_class_encoding_2.info()

In [ ]:
df_class_encoding_2.CLUSTER.value_counts()

In [ ]:
#Split
X3, y3 = df_class_encoding_2.drop(['CLUSTER','REGIAO_anon_S10','VAR_RECEITA','FLG_CAMPANHA_MKT_B','FLG_CAMPANHA_MKT_D','FLG_CAMPANHA_MKT_A','CANAL_anon_S7','CANAL_anon_S0','FLG_CAMPANHA_MKT_C','MARCA_1','MARCA_2','MARCA_5','FLG_CAMPANHA_MKT_E'],axis=1), df_class_encoding_2['CLUSTER']

print("X3", X3.shape)
print("y3", y3.shape)

# Usar train_test_split para criar o conjunto de teste
X3_training, X3_test, y3_training, y3_test = train_test_split(X3, y3, random_state=seed, test_size=0.25, stratify=y)

# Como saída, temos a porção de treino e teste para as variáveis X e Y
print("Test set X", X3_test.shape)
print("Test set y", y3_test.shape)

In [ ]:
# Random Forest Classifier com validação cruzada e GridSearch - modelo 3

# Definir os possiveis valores para cada hiperparametro para serem explorados
params3 = {'max_depth': [2, 3, 4, 5, 6],
          'min_samples_split': [2, 3, 4, 5, 6],
          'min_samples_leaf': [1, 2, 3]}

# Criar objeto com o RandomForestClassifier
modelo_3 = RandomForestClassifier(criterion='entropy',random_state=seed)

# Criar objeto KFold com StratifiedKFold para validação cruzada
skf3 = StratifiedKFold(n_splits=3, shuffle=True, random_state=seed)

# Criar um objeto de grade de busca com GridSearchCV, a partir dos parâmetros que definimos
grid_search_3 = GridSearchCV(modelo_3, param_grid=params3, scoring='roc_auc',
                           cv=skf3.split(X3_training, y3_training))

# Treinar o modelo com o grid search
grid_search_3.fit(X3_training, y3_training)

# Printar a melhor combinação de hiperparâmetros
print('Melhores hiperparâmetros:')
print(grid_search_3.best_params_)

#Melhores hiperparâmetros:
#{'max_depth': 6, 'min_samples_leaf': 1, 'min_samples_split': 2}

In [ ]:
# Gera um modelo com as melhores combinações de hiperparâmetros
modelo_3 = RandomForestClassifier(criterion='entropy',random_state=seed,
                                max_depth=grid_search_3.best_params_['max_depth'],
                                min_samples_split=grid_search_3.best_params_['min_samples_split'],
                                min_samples_leaf=grid_search_3.best_params_['min_samples_leaf'])

# Fita o modelo com o dataset de treino
modelo_3.fit(X3_training, y3_training)

# Usa o modelo para prever o conjunto de teste
y_pred_3 = modelo_3.predict(X3_test)

In [ ]:
# Calcular e printar métricas

# métricas
print("Acurácia no conjunto de treino: {:.3f}".format(modelo_3.score(X3_training, y3_training)))
print("Acurácia no conjunto de teste: {:.3f}".format(modelo_3.score(X3_test, y3_test)))

# O classification report traz diversas métricas interessantes, en formato de tabela, de forma simples
print(classification_report(y3_test, y_pred_3))

# A matriz de confusão mostra de forma fácil os dados
# Verdadeiros Positivos | Falsos Positivos
# Falsos Negativos | Verdadeiros Negativos
print(confusion_matrix(y3_test, y_pred_3))
print("Área sob a curva ROC: {:.3f}".format(roc_auc_score(y3_test, y_pred_3)))

# Plotar importância das features
# Avalia quão importante é cada variável para as decisões que a árvore fornece

feature_scores_3 = pd.Series(modelo_3.feature_importances_, index=X3.columns).sort_values(ascending=False)
print(feature_scores_3)

# plot
sns.barplot(x=feature_scores_3, y=feature_scores_3.index)

# labels
plt.xlabel('Importância das variáveis')
plt.ylabel('Variáveis')

# título
plt.title("Importância das variáveis")

# visualização
plt.show()

#### 1.5.6 Feature selection com RFE (Eliminação recursiva)

In [ ]:
df_class_encoding_2.info()

In [ ]:
#Split
Xrfe, yrfe = df_class_encoding_2.drop(['CLUSTER'],axis=1), df_class_encoding_2['CLUSTER']

print("Xrfe", Xrfe.shape)
print("yrfe", yrfe.shape)

# Usar train_test_split para criar o conjunto de teste
Xrfe_training, Xrfe_test, yrfe_training, yrfe_test = train_test_split(Xrfe, yrfe, random_state=seed, test_size=0.25, stratify=y)

# Como saída, temos a porção de treino e teste para as variáveis X e Y
print("Test set Xrfe", Xrfe_test.shape)
print("Test set yrfe", yrfe_test.shape)

In [ ]:
#RFE
estimator = RandomForestClassifier(criterion='entropy',random_state=seed)
selector = RFE(estimator,  step=1)
selector = selector.fit(Xrfe_training, yrfe_training)
print(selector.support_)
print(selector.ranking_)

In [ ]:
df_class_encoding_2.drop(['CLUSTER'],axis=1).info()

#### 1.5.7 Classificação - Modelo 4

In [ ]:
#Split
X4, y4 = df_class_encoding_2.drop(['CLUSTER','FLG_CAMPANHA_MKT_B','REGIAO_anon_S10','CANAL_anon_S7','FLG_CAMPANHA_MKT_D','FLG_CAMPANHA_MKT_A','CANAL_anon_S0','FLG_CAMPANHA_MKT_C','MARCA_1','MARCA_2','MARCA_5','FLG_CAMPANHA_MKT_E'],axis=1), df_class_encoding_2.CLUSTER

print("X4", X4.shape)
print("y4", y4.shape)

# Usar train_test_split para criar o conjunto de test
X4_training, X4_test, y4_training, y4_test = train_test_split(X4, y4, random_state=seed, test_size=0.25, stratify=y)

# Como saída, temos a porção de treino e teste para as variáveis X e Y
print("Test set X", X4_test.shape)
print("Test set y", y4_test.shape)

In [ ]:
# Criar um objeto XGBClassifier
modelo_4 = xgb.XGBClassifier(objective="binary:logistic", random_state=seed, eval_metric="auc",use_label_encoder=False)

# Treinar o modelo usando o conjunto de treino
modelo_4.fit(X4_training, y4_training)

# Prever todos os resultados no dataset de teste
y_pred_4 = modelo_4.predict(X4_test)

In [ ]:
# Calcular e printar métricas
print("Acurácia no conjunto de treino: {:.3f}".format(modelo_4.score(X4_training, y4_training)))
print("Acurácia no conjunto de teste: {:.3f}".format(modelo_4.score(X4_test, y4_test)))

# O classification report traz diversas métricas interessantes, en formato de tabela, de forma simples
print(classification_report(y4_test, y_pred_4.ravel()))

# A matriz de confusão mostra de forma fácil os dados
# Verdadeiros Positivos | Falsos Positivos
# Falsos Negativos | Verdadeiros Negativos
print(confusion_matrix(y4_test, y_pred_4.ravel()))
print("Área sob a curva ROC: {:.3f}".format(roc_auc_score(y4_test, y_pred_4.ravel())))

# Plotar importância das features
# Avalia quão importante é cada variável para as decisões que a árvore fornece

feature_scores_4 = pd.Series(modelo_4.feature_importances_, index=X4.columns).sort_values(ascending=False)
print(feature_scores_4)

# plot
sns.barplot(x=feature_scores_4, y=feature_scores_4.index)

# labels
plt.xlabel('Importância das variáveis')
plt.ylabel('Variáveis')

# título
plt.title("Importância das variáveis")

# visualização
plt.show()

#### 1.5.8 Classificação - Modelo 5

In [ ]:
#Split
X5, y5 = df_class_encoding_2.drop(['CLUSTER','FLG_CAMPANHA_MKT_B','REGIAO_anon_S10','CANAL_anon_S7','FLG_CAMPANHA_MKT_D','FLG_CAMPANHA_MKT_A','CANAL_anon_S0','FLG_CAMPANHA_MKT_C','MARCA_1','MARCA_2','MARCA_5','FLG_CAMPANHA_MKT_E'],axis=1), df_class_encoding_2.CLUSTER

print("X5", X5.shape)
print("y5", y5.shape)

# Usar train_test_split para criar o conjunto de teste
X5_training, X5_test, y5_training, y5_test = train_test_split(X5, y5, random_state=seed, test_size=0.25, stratify=y)

# Como saída, temos a porção de treino e teste para as variáveis X e Y
print("Test set X", X5_test.shape)
print("Test set y", y5_test.shape)

In [ ]:
# XGBoost com Cross Validation e Grid search

# Define um conjunto de possíveis valores para os hiperparâmetros
params_5 = {'learning_rate': [0.1, 0.2, 0.3],
          'alpha': [5, 10, 15],
          'colsample_bytree': [0.1, 0.3, 0.5, 0.7, 1.0],
          'max_depth': [3, 4, 5]}

# Cria o modelo
modelo_5 = xgb.XGBClassifier(objective="binary:logistic", random_state=seed,
                                    eval_metric="auc", n_estimators=10,use_label_encoder=False)

# Cria o grid search com validação cruzada estratificada
skf_5 = StratifiedKFold(n_splits=3, shuffle=True, random_state=seed)
grid_search_5 = GridSearchCV(modelo_5, param_grid=params_5, scoring='roc_auc',
                           cv=skf_5.split(X5_training, y5_training))

# Treina o modelo e printa os melhores hiperparâmetros
grid_search_5.fit(X5_training, y5_training)
print('\n Melhores hiperparâmetros:')
print(grid_search_5.best_params_)

#Melhores hiperparâmetros:
#{'alpha': 5, 'colsample_bytree': 0.7, 'learning_rate': 0.3, 'max_depth': 5}

In [ ]:
# Atribui os melhores valores para os hiperparâmetros
modelo_5.set_params(learning_rate = grid_search_5.best_params_['learning_rate'],
                           alpha = grid_search_5.best_params_['alpha'],
                           colsample_bytree = grid_search_5.best_params_['colsample_bytree'],
                           max_depth = grid_search_5.best_params_['max_depth'])

# Treina o modelo usando os melhores hiperparâmetros
modelo_5.fit(X5_training, y5_training)

# Prever todos os resultados no dataset de teste
y_pred_5 = modelo_5.predict(X5_test)

In [ ]:
# Calcular e printar métricas
print("Acurácia no conjunto de treino: {:.3f}".format(modelo_5.score(X5_training, y5_training)))
print("Acurácia no conjunto de teste: {:.3f}".format(modelo_5.score(X5_test, y5_test)))

# O classification report traz diversas métricas interessantes, en formato de tabela, de forma simples
print(classification_report(y5_test, y_pred_5))

# A matriz de confusão mostra de forma fácil os dados
# Verdadeiros Positivos | Falsos Positivos
# Falsos Negativos | Verdadeiros Negativos
print(confusion_matrix(y5_test, y_pred_5))
print("Área sob a curva ROC: {:.3f}".format(roc_auc_score(y5_test, y_pred_5)))

# Plotar importância das features
# Avalia quão importante é cada variável para as decisões que o modelo fornece

feature_scores_5 = pd.Series(modelo_5.feature_importances_, index=X5.columns).sort_values(ascending=False)
print(feature_scores_5)

#plot
sns.barplot(x=feature_scores_5, y=feature_scores_5.index)

#labels
plt.xlabel('Importância das variáveis')
plt.ylabel('Variáveis')

# título
plt.title("Importância das variáveis")

# visualização
plt.show()

#### 1.5.9 Comparação entre os modelos de classificação

In [ ]:
print("Modelo 1 - Acurácia no conjunto de teste: {:.3f}".format(modelo_1.score(X_test, y_test)))
print("Modelo 2 - Acurácia no conjunto de teste: {:.3f}".format(modelo_2.score(X2_test, y2_test)))
print("Modelo 3 - Acurácia no conjunto de teste: {:.3f}".format(modelo_3.score(X3_test, y3_test)))
print("Modelo 4 - Acurácia no conjunto de teste: {:.3f}".format(modelo_4.score(X4_test, y4_test)))
print("Modelo 5 - Acurácia no conjunto de teste: {:.3f}".format(modelo_5.score(X5_test, y5_test)))

In [ ]:
print("Modelo 1",classification_report(y_test, y_pred_1))
print("Modelo 2",classification_report(y2_test, y_pred_2))
print("Modelo 3",classification_report(y3_test, y_pred_3))
print("Modelo 4",classification_report(y4_test, y_pred_4))
print("Modelo 5",classification_report(y5_test, y_pred_5))

#### 1.5.10 Resultados da classificação

Com o resultado do cluster foi possível verificar que o grupo mais relevante para alavancagem de vendas é cluster 1, sendo assim iniciou-se uma tarefa de aprendizagem supervisonada para classificar os produtos em pertencentes ao cluster 1 (1)  e não pertencente ao cluster 1 (0). Esse modelo é importante para classificar novos materiais de acordo com o desempenho das variáveis históricas. Além disso, a classificação desses materiais traz a vantagem do conhecimento prévio do comportamento do material,ou seja, quais canais,regiões, marcas, categorias, campanhas e outras variáveis com as quais o material performa melhor em venda.

O modelo escolhido foi o modelo 4. O modelo 4 (XGBClassifier) foi treinado com as variáveis selecionadas pelo método RFE,por isso, é um modelo menos complexo com 11 variáveis. Apresenta a segunda maior acuracidade (0.968) e o segundo maior recall (0.67). 

Apesar do modelo 1 apresentar maior acurácia e recall que o modelo 4, seu conjunto de dados dificulta a interpretação dos resultados, visto que é um modelo com mais de 100 variáveis.

### 1.6 Previsão da quantidade de venda bruta - Regressão

#### 1.6.1 Preparação da base de dados - Modelos 6, 7 e 8

In [ ]:
base_previsao = base

In [ ]:
base_previsao.info()

In [ ]:
#filtrar somente o cluster 1
aux_mat = df_cluster[df_cluster['CLUSTER'] == 1]
aux_mat =  aux_mat['COD_MATERIAL'].unique()
print(len(aux_mat))

base_previsao_2 = base_previsao[base_previsao['COD_MATERIAL'].isin(aux_mat)]

In [ ]:
base_previsao_2.info()

In [ ]:
def extrair_campanha(arg1):
    return arg1[-2:]

base_previsao_2['N_CAMPANHA'] = base_previsao_2.COD_CICLO.apply(extrair_campanha)

In [ ]:
base_previsao_2['N_CAMPANHA'] = base_previsao_2['N_CAMPANHA'].astype('int64')

In [ ]:
base_previsao_2.N_CAMPANHA.unique()

In [ ]:
base_previsao_2

In [ ]:
#COD_MATERIAL
df_material_qtd = base_previsao_2.groupby(['COD_MATERIAL','COD_CICLO'], as_index=False)['QT_VENDA_BRUTO'].mean()
df_material_qtd  = df_material_qtd.set_axis(['COD_MATERIAL','COD_CICLO','MATERIAL_MEDIA_QTD'], axis=1, inplace=False)
df_material_qtd  = df_material_qtd.sort_values(by=['MATERIAL_MEDIA_QTD'],ascending=False)
df_material_qtd

base_previsao_2 = pd.merge(base_previsao_2,df_material_qtd,on= ['COD_MATERIAL','COD_CICLO'], how='left' )
base_previsao_2

In [ ]:
#COD_CANAL
df_canal_qtd = base_previsao_2.groupby(['COD_CANAL','COD_CICLO'], as_index=False)['QT_VENDA_BRUTO'].mean()
df_canal_qtd  = df_canal_qtd.set_axis(['COD_CANAL','COD_CICLO','CANAL_MEDIA_QTD'], axis=1, inplace=False)
df_canal_qtd  = df_canal_qtd.sort_values(by=['CANAL_MEDIA_QTD'],ascending=False)
df_canal_qtd

base_previsao_2 = pd.merge(base_previsao_2,df_canal_qtd,on= ['COD_CANAL','COD_CICLO'], how='left' )
base_previsao_2

In [ ]:
#'DES_CATEGORIA_MATERIAL'
df_categoria_qtd = base_previsao_2.groupby(['DES_CATEGORIA_MATERIAL','COD_CICLO'], as_index=False)['QT_VENDA_BRUTO'].mean()
df_categoria_qtd  = df_categoria_qtd.set_axis(['DES_CATEGORIA_MATERIAL','COD_CICLO','CATEGORIA_MEDIA_QTD'], axis=1, inplace=False)
df_categoria_qtd  = df_categoria_qtd.sort_values(by=['CATEGORIA_MEDIA_QTD'],ascending=False)
df_categoria_qtd

base_previsao_2 = pd.merge(base_previsao_2,df_categoria_qtd,on= ['DES_CATEGORIA_MATERIAL','COD_CICLO'], how='left' )
base_previsao_2

In [ ]:
#'DES_MARCA_MATERIAL'
df_marca_qtd = base_previsao_2.groupby(['DES_MARCA_MATERIAL','COD_CICLO'], as_index=False)['QT_VENDA_BRUTO'].mean()
df_marca_qtd= df_marca_qtd.set_axis(['DES_MARCA_MATERIAL','COD_CICLO','MARCA_MEDIA_QTD'], axis=1, inplace=False)
df_marca_qtd = df_marca_qtd.sort_values(by=['MARCA_MEDIA_QTD'],ascending=False)
df_marca_qtd

base_previsao_2 = pd.merge(base_previsao_2,df_marca_qtd,on= ['DES_MARCA_MATERIAL','COD_CICLO'], how='left' )
base_previsao_2

In [ ]:
#COD_REGIAO
df_regiao_qtd = base_previsao_2.groupby(['COD_REGIAO','COD_CICLO'], as_index=False)['QT_VENDA_BRUTO'].mean()
df_regiao_qtd = df_regiao_qtd.set_axis(['COD_REGIAO','COD_CICLO','REGIAO_MEDIA_QTD'], axis=1, inplace=False)
df_regiao_qtd = df_regiao_qtd.sort_values(by=['REGIAO_MEDIA_QTD'],ascending=False)
df_regiao_qtd 

base_previsao_2 = pd.merge(base_previsao_2,df_regiao_qtd,on= ['COD_REGIAO','COD_CICLO'], how='left' )
base_previsao_2

In [ ]:
#FLG_CAMPANHA_MKT_B
df_flagb_qtd = base_previsao_2.groupby(['FLG_CAMPANHA_MKT_B','COD_CICLO'], as_index=False)['QT_VENDA_BRUTO'].mean()
df_flagb_qtd = df_flagb_qtd[df_flagb_qtd['FLG_CAMPANHA_MKT_B'] == 1]
df_flagb_qtd = df_flagb_qtd.set_axis(['FLG_CAMPANHA_MKT_B','COD_CICLO','FLAGB_MEDIA_QTD'], axis=1, inplace=False)
df_flagb_qtd = df_flagb_qtd.sort_values(by=['COD_CICLO'])
df_flagb_qtd

base_previsao_2 = pd.merge(base_previsao_2,df_flagb_qtd[['COD_CICLO','FLAGB_MEDIA_QTD']],on= ['COD_CICLO'], how='left' )
base_previsao_2

In [ ]:
#FLG_CAMPANHA_MKT_A
df_flaga_qtd = base_previsao_2.groupby(['FLG_CAMPANHA_MKT_A','COD_CICLO'], as_index=False)['QT_VENDA_BRUTO'].mean()
df_flaga_qtd = df_flaga_qtd[df_flaga_qtd['FLG_CAMPANHA_MKT_A'] == 1]
df_flaga_qtd = df_flaga_qtd.set_axis(['FLG_CAMPANHA_MKT_A','COD_CICLO','FLAGA_MEDIA_QTD'], axis=1, inplace=False)
df_flaga_qtd = df_flaga_qtd.sort_values(by=['COD_CICLO'])
df_flaga_qtd

base_previsao_2 = pd.merge(base_previsao_2,df_flaga_qtd[['COD_CICLO','FLAGA_MEDIA_QTD']],on= ['COD_CICLO'], how='left' )
base_previsao_2

In [ ]:
base_previsao_2.info()

In [ ]:
base_previsao_2.head()

In [ ]:
base_previsao_2 = base_previsao_2.drop(['COD_CICLO','COD_MATERIAL','QT_DEVOLUCAO','VL_RECEITA_BRUTA','VL_RECEITA_LIQUIDA','ANO','FLG_CAMPANHA_MKT_A','FLG_CAMPANHA_MKT_B','FLG_CAMPANHA_MKT_C','FLG_CAMPANHA_MKT_D','FLG_CAMPANHA_MKT_E','COD_CANAL','DES_CATEGORIA_MATERIAL','DES_MARCA_MATERIAL','COD_REGIAO'],axis=1)

In [ ]:
base_previsao_2 = base_previsao_2.fillna(0)
base_previsao_2

In [ ]:
base_previsao_2.info()

In [ ]:
base_previsao_2 = base_previsao_2[['QT_VENDA_BRUTO','N_CAMPANHA','FLG_DATA','MATERIAL_MEDIA_QTD','CANAL_MEDIA_QTD','CATEGORIA_MEDIA_QTD','MARCA_MEDIA_QTD','REGIAO_MEDIA_QTD','FLAGB_MEDIA_QTD','FLAGA_MEDIA_QTD','PCT_DESCONTO','VL_PRECO']]

In [ ]:
base_previsao_2.info()

In [ ]:
plt.figure(figsize=(10,5))

sns.heatmap(base_previsao_2.corr(),cmap='coolwarm')

#### 1.6.2 Regressão - Modelo 6 

In [ ]:
# Split dos dados de treino e teste

X6 = base_previsao_2.drop(['QT_VENDA_BRUTO'], axis=1)
y6 = base_previsao_2['QT_VENDA_BRUTO']

# Utilizando a função train_test_split para dividir os dados em treino e teste
X6_training, X6_test, y6_training, y6_test = train_test_split(X6, y6,
                                                          test_size=0.25, 
                                                          shuffle=False)

print("Conjunto de treino X6:", X6_training.shape)
print("Conjunto de treino y6:", y6_training.shape)
print("Conjunto de teste  X6:", X6_test.shape)
print("Conjunto de teste  y6:", y6_test.shape)

In [ ]:
# Random Forest Regressor - MODELO 6

# criterion default= 'squared_error'
modelo_6 = RandomForestRegressor(random_state=seed)

# Fitar, ou treinar, o modelo usando o dataset de treino
modelo_6.fit(X6_training, y6_training)

# Prever todos os resultados no dataset de teste
y_pred_6 = modelo_6.predict(X6_test)

# Calcular e printar métricas
print("Mean absolute error (MAE): {:.3f}".format(mean_absolute_error(y6_test, y_pred_6)))
print("Mean squared error (MSE): {:.3f}".format(mean_squared_error(y6_test, y_pred_6)))
print("Mean absolute percentage error(MAPE): {:.3f}".format(mean_absolute_percentage_error(y6_test, y_pred_6)))
print("R² Score: {:.3f}".format(r2_score(y6_test, y_pred_6)))

feature_scores_6 = pd.Series(modelo_6.feature_importances_, index=X6.columns).sort_values(ascending=False)
print(feature_scores_6)

# Creating a seaborn bar plot
sns.barplot(x=feature_scores_6, y=feature_scores_6.index)

# Add labels to the graph
plt.xlabel('Importância das variáveis')
plt.ylabel('Variáveis')

# Add title to the graph
plt.title("Importância das variáveis")

# Visualize the graph
plt.show()

In [ ]:
# Desenhando o gráfico de valores previstos por valores reais
plt.figure(figsize=(16,8))
plt.title('Quantidade de venda bruta - Previsto e Real',fontsize=20)
df_6 = pd.DataFrame({'Real':y6_test,'Previsto':y_pred_6})
#df_6.sort_values(by=['Real'],ascending=True,inplace=True)
df_6 = df_6.reset_index(drop=True)
plt.plot(df_6)
plt.legend(['Real','Previsto'],fontsize=20)
plt.ylabel('Quantidade de venda bruta',fontsize=20)
plt.xlabel('Observações',fontsize=20)
plt.show()

#### 1.6.3 Regressão - Modelo 7

In [ ]:
# Split dos dados de treino e teste

X7 = base_previsao_2.drop(['QT_VENDA_BRUTO','FLAGB_MEDIA_QTD','FLAGA_MEDIA_QTD','FLG_DATA','N_CAMPANHA'], axis=1)
y7 = base_previsao_2['QT_VENDA_BRUTO']

# Utilizando a função train_test_split para dividir os dados em treino e teste (2 pt)
X7_training, X7_test, y7_training, y7_test = train_test_split(X7, y7,
                                                          test_size=0.25, 
                                                          shuffle=False)

print("Conjunto de treino X7:", X7_training.shape)
print("Conjunto de treino y7:", y7_training.shape)
print("Conjunto de teste  X7:", X7_test.shape)
print("Conjunto de teste  y7:", y7_test.shape)

In [ ]:
# Random Forest Regressor - MODELO 7

# criterion default= 'squared_error'
modelo_7 = RandomForestRegressor(random_state=seed)

# Fitar, ou treinar, o modelo usando o dataset de treino
modelo_7.fit(X7_training, y7_training)

# Prever todos os resultados no dataset de teste
y_pred_7 = modelo_7.predict(X7_test)

# Calcular e printar métricas
print("Mean absolute error (MAE): {:.3f}".format(mean_absolute_error(y7_test, y_pred_7)))
print("Mean squared error (MSE): {:.3f}".format(mean_squared_error(y7_test, y_pred_7)))
print("Mean absolute percentage error(MAPE): {:.3f}".format(mean_absolute_percentage_error(y7_test, y_pred_7)))
print("R² Score: {:.3f}".format(r2_score(y7_test, y_pred_7)))

In [ ]:
feature_scores_7 = pd.Series(modelo_7.feature_importances_, index=X7.columns).sort_values(ascending=False)
print(feature_scores_7)

# Creating a seaborn bar plot
sns.barplot(x=feature_scores_7, y=feature_scores_7.index)

# Add labels to the graph
plt.xlabel('Importância das variáveis')
plt.ylabel('Variáveis')

# Add title to the graph
plt.title("Importância das variáveis")

# Visualize the graph
plt.show()

In [ ]:
# Desenhando o gráfico de valores previstos por valores reais
plt.figure(figsize=(16,8))
plt.title('Quantidade de venda bruta - Previsto e Real',fontsize=20)
df_7 = pd.DataFrame({'Real':y7_test,'Previsto':y_pred_7})
#df_6.sort_values(by=['Real'],ascending=True,inplace=True)
df_7 = df_7.reset_index(drop=True)
plt.plot(df_7)
plt.legend(['Real','Previsto'],fontsize=20)
plt.ylabel('Quantidade de venda bruta',fontsize=20)
plt.xlabel('Observações',fontsize=20)
plt.show()

#### 1.6.4 Regressão - Modelo 8

In [ ]:
# Split dos dados de treino e teste

X8 = base_previsao_2.drop(['QT_VENDA_BRUTO','FLAGB_MEDIA_QTD','FLAGA_MEDIA_QTD','FLG_DATA','N_CAMPANHA'], axis=1)
y8 = base_previsao_2['QT_VENDA_BRUTO']

# Utilizando a função train_test_split para dividir os dados em treino e teste (2 pt)
X8_training, X8_test, y8_training, y8_test = train_test_split(X8, y8,
                                                          test_size=0.25, 
                                                          shuffle=False)

print("Conjunto de treino X8:", X8_training.shape)
print("Conjunto de treino y8:", y8_training.shape)
print("Conjunto de teste  X8:", X8_test.shape)
print("Conjunto de teste  y8:", y8_test.shape)

In [ ]:
# Random Forest com seleção de variáveis e Grid Search

# definindo os valores possíveis dos parâmetros a serem testados
params_8 = {'n_estimators': [5, 50, 100, 500],
          'max_features': [2, 5, 9, 13],
          'max_depth': [2, 5, 10, 50],
          'min_samples_split': [2, 8, 15, 30],}

# criando o objeto do modelo com RandomForestRegressor
modelo_8 = RandomForestRegressor(random_state = seed)

# criando o objeto do grid search com GridSearchCV
grid_search_8 = GridSearchCV(modelo_8 , param_grid=params_8, return_train_score=True) # valor padrão para quebras é 3

# treinando o modelo com o grid search
grid_search_8.fit(X8_training, y8_training)

# imprimindo a melhor combinação de hiperparâmetros
print('\n Melhores hiperparâmetros:')
print(grid_search_8.best_params_)

# configurando o modelo com a melhor combinação de hiperparâmetros

modelo_8.set_params(n_estimators = grid_search_8.best_params_['n_estimators'],
                           max_features = grid_search_8.best_params_['max_features'],
                           max_depth = grid_search_8.best_params_['max_depth'],
                           min_samples_split = grid_search_8.best_params_['min_samples_split'])

# treinando um modelo com a melhor combinação de hiperparâmetros
modelo_8.fit(X8_training, y8_training)

# Prever todos os resultados no dataset de validação
y_pred_8 = modelo_8.predict(X8_test)

# Melhores hiperparâmetros:
#{max_depth': 50, 'max_features': 2, 'min_samples_split': 2, 'n_estimators': 500}

In [ ]:
# métricas
print("Mean absolute error (MAE): {:.3f}".format(mean_absolute_error(y8_test, y_pred_8)))
print("Mean squared error (MSE): {:.3f}".format(mean_squared_error(y8_test, y_pred_8)))
print("Mean absolute percentage error(MAPE): {:.3f}".format(mean_absolute_percentage_error(y8_test, y_pred_8)))
print("R² Score: {:.3f}".format(r2_score(y8_test, y_pred_8)))

feature_scores_8 = pd.Series(modelo_8.feature_importances_, index=X8.columns).sort_values(ascending=False)
print(feature_scores_8)

# Creating a seaborn bar plot
sns.barplot(x=feature_scores_8, y=feature_scores_8.index)

# Add labels to the graph
plt.xlabel('Importância das variáveis')
plt.ylabel('Variáveis')

# Add title to the graph
plt.title("Importância das variáveis")

# Visualize the graph
plt.show()

In [ ]:
# Desenhando o gráfico de valores previstos por valores reais
plt.figure(figsize=(16,8))
plt.title('Quantidade de venda bruta - Previsto e Real',fontsize=20)
df_8 = pd.DataFrame({'Real':y6_test,'Previsto':y_pred_8})
df_8 = df_8.reset_index(drop=True)
plt.plot(df_8)
plt.legend(['Real','Previsto'],fontsize=20)
plt.ylabel('Quantidade de venda bruta',fontsize=20)
plt.xlabel('Observações',fontsize=20)
plt.show()

#### 1.6.5 Comparando os modelos de previsão da quantidade bruta de vendas

In [ ]:
#modelo 6 - Random Forest com todas as variáveis

print("Modelo 6 - Mean absolute error (MAE): {:.3f}".format(mean_absolute_error(y6_test, y_pred_6)))
print("Modelo 6 - Mean squared error (MSE): {:.3f}".format(mean_squared_error(y6_test, y_pred_6)))
print("Modelo 6 - Mean absolute percentage error(MAPE): {:.3f}".format(mean_absolute_percentage_error(y6_test, y_pred_6)))
print("Modelo 6 - R² Score: {:.3f}".format(r2_score(y6_test, y_pred_6)))

#modelo 7 - Random Forest com seleção de variáveis

print("Modelo 7 - Mean absolute error (MAE): {:.3f}".format(mean_absolute_error(y7_test, y_pred_7)))
print("Modelo 7 - Mean squared error (MSE): {:.3f}".format(mean_squared_error(y7_test, y_pred_7)))
print("Modelo 7 - Mean absolute percentage error(MAPE): {:.3f}".format(mean_absolute_percentage_error(y7_test, y_pred_7)))
print("Modelo 7 - R² Score: {:.3f}".format(r2_score(y7_test, y_pred_7)))

#modelo 8 - Random Forest com seleção de variáveis e grid search

print("Modelo 8 - Mean absolute error (MAE): {:.3f}".format(mean_absolute_error(y8_test, y_pred_8)))
print("Modelo 8 - Mean squared error (MSE): {:.3f}".format(mean_squared_error(y8_test, y_pred_8)))
print("Modelo 8 - Mean absolute percentage error(MAPE): {:.3f}".format(mean_absolute_percentage_error(y8_test, y_pred_8)))
print("Modelo 8 - R² Score: {:.3f}".format(r2_score(y8_test, y_pred_8)))

#### 1.6.6 Resultado da previsão da quantidade bruta de venda

O modelo de previsão de vendas campeão foi o modelo 7 (Random Forest com feature selection).

O modelo 6 não foi escolhido pelo fato de não utilizar uma técnica de feature selection. E o modelo 8, mesmo com feature 
selection e grid search, não performou bem, apresentando um MAPE alto.

### 1.7 Previsão  da receita líquida de venda - Regressão

#### 1.7.1 Preparação da base de dados - Modelos 9, 10 e 11

In [ ]:
base_previsao_valor = base

In [ ]:
base_previsao_valor.info()

In [ ]:
#filtrar somente o cluster 1
aux_mat_2 = df_cluster[df_cluster['CLUSTER'] == 1]
aux_mat_2 =  aux_mat_2['COD_MATERIAL'].unique()
print(len(aux_mat_2))

base_previsao_valor_2 = base_previsao_valor[base_previsao_valor['COD_MATERIAL'].isin(aux_mat_2)]

In [ ]:
base_previsao_valor_2.info()

In [ ]:
def extrair_campanha(arg1):
    return arg1[-2:]

base_previsao_valor_2['N_CAMPANHA'] = base_previsao_valor_2.COD_CICLO.apply(extrair_campanha)

base_previsao_valor_2['N_CAMPANHA'] = base_previsao_valor_2['N_CAMPANHA'].astype('int64')

print(base_previsao_valor_2.N_CAMPANHA.unique())

base_previsao_valor_2

In [ ]:
#COD_MATERIAL
df_material_qtd_2 = base_previsao_valor_2.groupby(['COD_MATERIAL','COD_CICLO'], as_index=False)['VL_RECEITA_LIQUIDA'].mean()
df_material_qtd_2  = df_material_qtd_2.set_axis(['COD_MATERIAL','COD_CICLO','MATERIAL_MEDIA_VL'], axis=1, inplace=False)
df_material_qtd_2  = df_material_qtd_2.sort_values(by=['MATERIAL_MEDIA_VL'],ascending=False)
df_material_qtd_2

base_previsao_valor_2 = pd.merge(base_previsao_valor_2,df_material_qtd_2,on= ['COD_MATERIAL','COD_CICLO'], how='left' )
base_previsao_valor_2

#COD_CANAL
df_canal_qtd_2 = base_previsao_valor_2.groupby(['COD_CANAL','COD_CICLO'], as_index=False)['VL_RECEITA_LIQUIDA'].mean()
df_canal_qtd_2  = df_canal_qtd_2.set_axis(['COD_CANAL','COD_CICLO','CANAL_MEDIA_VL'], axis=1, inplace=False)
df_canal_qtd_2  = df_canal_qtd_2.sort_values(by=['CANAL_MEDIA_VL'],ascending=False)
df_canal_qtd_2

base_previsao_valor_2 = pd.merge(base_previsao_valor_2,df_canal_qtd_2,on= ['COD_CANAL','COD_CICLO'], how='left' )
base_previsao_valor_2

#'DES_CATEGORIA_MATERIAL'
df_categoria_qtd_2 = base_previsao_valor_2.groupby(['DES_CATEGORIA_MATERIAL','COD_CICLO'], as_index=False)['VL_RECEITA_LIQUIDA'].mean()
df_categoria_qtd_2  = df_categoria_qtd_2.set_axis(['DES_CATEGORIA_MATERIAL','COD_CICLO','CATEGORIA_MEDIA_VL'], axis=1, inplace=False)
df_categoria_qtd_2  = df_categoria_qtd_2.sort_values(by=['CATEGORIA_MEDIA_VL'],ascending=False)
df_categoria_qtd_2

base_previsao_valor_2 = pd.merge(base_previsao_valor_2,df_categoria_qtd_2,on= ['DES_CATEGORIA_MATERIAL','COD_CICLO'], how='left' )
base_previsao_valor_2

#'DES_MARCA_MATERIAL'
df_marca_qtd_2 = base_previsao_valor_2.groupby(['DES_MARCA_MATERIAL','COD_CICLO'], as_index=False)['VL_RECEITA_LIQUIDA'].mean()
df_marca_qtd_2= df_marca_qtd_2.set_axis(['DES_MARCA_MATERIAL','COD_CICLO','MARCA_MEDIA_VL'], axis=1, inplace=False)
df_marca_qtd_2 = df_marca_qtd_2.sort_values(by=['MARCA_MEDIA_VL'],ascending=False)
df_marca_qtd_2

base_previsao_valor_2 = pd.merge(base_previsao_valor_2,df_marca_qtd_2,on= ['DES_MARCA_MATERIAL','COD_CICLO'], how='left' )
base_previsao_valor_2

#COD_REGIAO
df_regiao_qtd_2 = base_previsao_valor_2.groupby(['COD_REGIAO','COD_CICLO'], as_index=False)['VL_RECEITA_LIQUIDA'].mean()
df_regiao_qtd_2 = df_regiao_qtd_2.set_axis(['COD_REGIAO','COD_CICLO','REGIAO_MEDIA_VL'], axis=1, inplace=False)
df_regiao_qtd_2 = df_regiao_qtd_2.sort_values(by=['REGIAO_MEDIA_VL'],ascending=False)
df_regiao_qtd_2 

base_previsao_valor_2 = pd.merge(base_previsao_valor_2,df_regiao_qtd_2,on= ['COD_REGIAO','COD_CICLO'], how='left' )
base_previsao_valor_2

#FLG_CAMPANHA_MKT_B
df_flagb_qtd_2 = base_previsao_valor_2.groupby(['FLG_CAMPANHA_MKT_B','COD_CICLO'], as_index=False)['VL_RECEITA_LIQUIDA'].mean()
df_flagb_qtd_2 = df_flagb_qtd_2[df_flagb_qtd_2['FLG_CAMPANHA_MKT_B'] == 1]
df_flagb_qtd_2 = df_flagb_qtd_2.set_axis(['FLG_CAMPANHA_MKT_B','COD_CICLO','FLAGB_MEDIA_VL'], axis=1, inplace=False)
df_flagb_qtd_2 = df_flagb_qtd_2.sort_values(by=['COD_CICLO'])
df_flagb_qtd_2

base_previsao_valor_2 = pd.merge(base_previsao_valor_2,df_flagb_qtd_2[['COD_CICLO','FLAGB_MEDIA_VL']],on= ['COD_CICLO'], how='left' )
base_previsao_valor_2

#FLG_CAMPANHA_MKT_A
df_flaga_qtd_2 = base_previsao_valor_2.groupby(['FLG_CAMPANHA_MKT_A','COD_CICLO'], as_index=False)['VL_RECEITA_LIQUIDA'].mean()
df_flaga_qtd_2 = df_flaga_qtd_2[df_flaga_qtd_2['FLG_CAMPANHA_MKT_A'] == 1]
df_flaga_qtd_2 = df_flaga_qtd_2.set_axis(['FLG_CAMPANHA_MKT_A','COD_CICLO','FLAGA_MEDIA_VL'], axis=1, inplace=False)
df_flaga_qtd_2 = df_flaga_qtd_2.sort_values(by=['COD_CICLO'])
df_flaga_qtd_2

base_previsao_valor_2 = pd.merge(base_previsao_valor_2,df_flaga_qtd_2[['COD_CICLO','FLAGA_MEDIA_VL']],on= ['COD_CICLO'], how='left' )
base_previsao_valor_2

In [ ]:
base_previsao_valor_2.info()

In [ ]:
base_previsao_valor_2.head()

In [ ]:
base_previsao_valor_2 = base_previsao_valor_2.drop(['COD_CICLO','COD_MATERIAL','QT_DEVOLUCAO','QT_VENDA_BRUTO','VL_RECEITA_BRUTA','ANO','FLG_CAMPANHA_MKT_A','FLG_CAMPANHA_MKT_B','FLG_CAMPANHA_MKT_C','FLG_CAMPANHA_MKT_D','FLG_CAMPANHA_MKT_E','COD_CANAL','DES_CATEGORIA_MATERIAL','DES_MARCA_MATERIAL','COD_REGIAO'],axis=1)

In [ ]:
base_previsao_valor_2 = base_previsao_valor_2.fillna(0)
base_previsao_valor_2

In [ ]:
base_previsao_valor_2.info()

In [ ]:
base_previsao_valor_2 = base_previsao_valor_2[['VL_RECEITA_LIQUIDA','N_CAMPANHA','FLG_DATA','MATERIAL_MEDIA_VL','CANAL_MEDIA_VL','CATEGORIA_MEDIA_VL','MARCA_MEDIA_VL','REGIAO_MEDIA_VL','FLAGB_MEDIA_VL','FLAGA_MEDIA_VL','PCT_DESCONTO','VL_PRECO']]

In [ ]:
base_previsao_valor_2.info()

In [ ]:
#matriz de correlação

plt.figure(figsize=(10,5))
sns.heatmap(base_previsao_valor_2.corr(),cmap='coolwarm')

#### 1.7.2 Regressão - Modelo 9

In [ ]:
# Split dos dados de treino e teste

X9 = base_previsao_valor_2.drop(['VL_RECEITA_LIQUIDA'], axis=1)
y9 = base_previsao_valor_2['VL_RECEITA_LIQUIDA']

# Utilizando a função train_test_split para dividir os dados em treino e teste
X9_training, X9_test, y9_training, y9_test = train_test_split(X9, y9,
                                                          test_size=0.25, 
                                                          shuffle=False)

print("Conjunto de treino X9:", X9_training.shape)
print("Conjunto de treino y9:", y9_training.shape)
print("Conjunto de teste  X9:", X9_test.shape)
print("Conjunto de teste  y9:", y9_test.shape)

In [ ]:
# Random Forest Regressor - MODELO 9

# criterion default= 'squared_error'
modelo_9 = RandomForestRegressor(random_state=seed)

# Fitar, ou treinar, o modelo usando o dataset de treino
modelo_9.fit(X9_training, y9_training)

# Prever todos os resultados no dataset de teste
y_pred_9 = modelo_9.predict(X9_test)

# Calcular e printar métricas
print("Mean absolute error (MAE): {:.3f}".format(mean_absolute_error(y9_test, y_pred_9)))
print("Mean squared error (MSE): {:.3f}".format(mean_squared_error(y9_test, y_pred_9)))
print("Mean absolute percentage error(MAPE): {:.3f}".format(mean_absolute_percentage_error(y9_test, y_pred_9)))
print("R² Score: {:.3f}".format(r2_score(y9_test, y_pred_9)))

feature_scores_9 = pd.Series(modelo_9.feature_importances_, index=X9.columns).sort_values(ascending=False)
print(feature_scores_9)

# Creating a seaborn bar plot
sns.barplot(x=feature_scores_9, y=feature_scores_9.index)

# Add labels to the graph
plt.xlabel('Importância das variáveis')
plt.ylabel('Variáveis')

# Add title to the graph
plt.title("Importância das variáveis")

# Visualize the graph
plt.show()

In [ ]:
# Desenhando o gráfico de valores previstos por valores reais
plt.figure(figsize=(16,8))
plt.title('Valor de receita líquida - Previsto e Real',fontsize=20)
df_9 = pd.DataFrame({'Real':y9_test,'Previsto':y_pred_9})
df_9 = df_9.reset_index(drop=True)
plt.plot(df_9)
plt.legend(['Real','Previsto'],fontsize=20)
plt.ylabel('Valor de receita líquida',fontsize=20)
plt.xlabel('Observações',fontsize=20)
plt.show()

#### 1.7.3 Regressão - Modelo 10

In [ ]:
# Split dos dados de treino e teste

X10 = base_previsao_valor_2.drop(['VL_RECEITA_LIQUIDA','FLAGB_MEDIA_VL','FLAGA_MEDIA_VL','FLG_DATA','N_CAMPANHA'], axis=1)
y10 = base_previsao_valor_2['VL_RECEITA_LIQUIDA']

# Utilizando a função train_test_split para dividir os dados em treino e teste 
X10_training, X10_test, y10_training, y10_test = train_test_split(X10, y10,
                                                          test_size=0.25, 
                                                          shuffle=False)
print("Conjunto de treino X10:", X10_training.shape)
print("Conjunto de treino y10:", y10_training.shape)
print("Conjunto de teste  X10:", X10_test.shape)
print("Conjunto de teste  y10:", y10_test.shape)

In [ ]:
# Random Forest Regressor - MODELO 10

# criterion default= 'squared_error'
modelo_10 = RandomForestRegressor(random_state=seed)

# Fitar, ou treinar, o modelo usando o dataset de treino
modelo_10.fit(X10_training, y10_training)

# Prever todos os resultados no dataset de teste
y_pred_10 = modelo_10.predict(X10_test)

# Calcular e printar métricas
print("Mean absolute error (MAE): {:.3f}".format(mean_absolute_error(y10_test, y_pred_10)))
print("Mean squared error (MSE): {:.3f}".format(mean_squared_error(y10_test, y_pred_10)))
print("Mean absolute percentage error(MAPE): {:.3f}".format(mean_absolute_percentage_error(y10_test, y_pred_10)))
print("R² Score: {:.3f}".format(r2_score(y10_test, y_pred_10)))

feature_scores_10 = pd.Series(modelo_10.feature_importances_, index=X10.columns).sort_values(ascending=False)
print(feature_scores_10)

# Creating a seaborn bar plot
sns.barplot(x=feature_scores_10, y=feature_scores_10.index)

# Add labels to the graph
plt.xlabel('Importância das variáveis')
plt.ylabel('Variáveis')

# Add title to the graph
plt.title("Importância das variáveis")

In [ ]:
# Desenhando o gráfico de valores previstos por valores reais
plt.figure(figsize=(16,8))
plt.title('Valor de receita líquida - Previsto e Real',fontsize=20)
df_10 = pd.DataFrame({'Real':y10_test,'Previsto':y_pred_10})
df_10 = df_10.reset_index(drop=True)
plt.plot(df_10)
plt.legend(['Real','Previsto'],fontsize=20)
plt.ylabel('Valor de receita líquida',fontsize=20)
plt.xlabel('Observações',fontsize=20)
plt.show()

#### 1.7.4 Regressão - Modelo 11

In [ ]:
# Split dos dados de treino e teste

X11 = base_previsao_valor_2.drop(['VL_RECEITA_LIQUIDA','FLAGB_MEDIA_VL','FLAGA_MEDIA_VL','FLG_DATA','N_CAMPANHA'], axis=1)
y11 = base_previsao_valor_2['VL_RECEITA_LIQUIDA']

# Utilizando a função train_test_split para dividir os dados em treino e teste
X11_training, X11_test, y11_training, y11_test = train_test_split(X11, y11,
                                                          test_size=0.25, 
                                                          shuffle=False)

print("Conjunto de treino X11:", X11_training.shape)
print("Conjunto de treino y11:", y11_training.shape)
print("Conjunto de teste  X11:", X11_test.shape)
print("Conjunto de teste  y11:", y11_test.shape)

In [ ]:
# Random Forest com seleção de variáveis e Grid Search

# definindo os valores possíveis dos parâmetros a serem testados
params_11 = {'n_estimators': [5, 50, 100, 500],
          'max_features': [2, 5, 9, 13],
          'max_depth': [2, 5, 10, 50],
          'min_samples_split': [2, 8, 15, 30],}

# criando o objeto do modelo com RandomForestRegressor
modelo_11 = RandomForestRegressor(random_state = seed)

# criando o objeto do grid search com GridSearchCV
grid_search_11 = GridSearchCV(modelo_11 , param_grid=params_11, return_train_score=True) # valor padrão para quebras é 3

# treinando o modelo com o grid search
grid_search_11.fit(X11_training, y11_training)

# imprimindo a melhor combinação de hiperparâmetros
print('\n Melhores hiperparâmetros:')
print(grid_search_11.best_params_)

# configurando o modelo com a melhor combinação de hiperparâmetros
modelo_11.set_params(n_estimators = grid_search_11.best_params_['n_estimators'],
                           max_features = grid_search_11.best_params_['max_features'],
                           max_depth = grid_search_11.best_params_['max_depth'],
                           min_samples_split = grid_search_11.best_params_['min_samples_split'])

# treinando um modelo com a melhor combinação de hiperparâmetros
modelo_11.fit(X11_training, y11_training)

# Prever todos os resultados no dataset de validação
y_pred_11 = modelo_11.predict(X11_test)

# Melhores hiperparâmetros:
#{'max_depth': 50, 'max_features': 5, 'min_samples_split': 2, 'n_estimators': 50}

In [ ]:
# métricas
print("Mean absolute error (MAE): {:.3f}".format(mean_absolute_error(y11_test, y_pred_11)))
print("Mean squared error (MSE): {:.3f}".format(mean_squared_error(y11_test, y_pred_11)))
print("Mean absolute percentage error(MAPE): {:.3f}".format(mean_absolute_percentage_error(y11_test, y_pred_11)))
print("R² Score: {:.3f}".format(r2_score(y11_test, y_pred_11)))

feature_scores_11 = pd.Series(modelo_11.feature_importances_, index=X11.columns).sort_values(ascending=False)
print(feature_scores_11)

# Creating a seaborn bar plot
sns.barplot(x=feature_scores_11, y=feature_scores_11.index)

# Add labels to the graph
plt.xlabel('Importância das variáveis')
plt.ylabel('Variáveis')

# Add title to the graph
plt.title("Importância das variáveis")

# Visualize the graph
plt.show()

In [ ]:
# Desenhando o gráfico de valores previstos por valores reais
plt.figure(figsize=(16,8))
plt.title('Valor de receita líquida - Previsto e Real',fontsize=20)
df_11 = pd.DataFrame({'Real':y11_test,'Previsto':y_pred_11})
df_11 = df_11.reset_index(drop=True)
plt.plot(df_11)
plt.legend(['Real','Previsto'],fontsize=20)
plt.ylabel('Valor de receita líquida',fontsize=20)
plt.xlabel('Observações',fontsize=20)
plt.show()

#### 1.7.5 Comparando os modelos de previsão da receita líquida de vendas

In [ ]:
#modelo 9 - Random Forest com todas as variáveis

print("Modelo 9 - Mean absolute error (MAE): {:.3f}".format(mean_absolute_error(y9_test, y_pred_9)))
print("Modelo 9 - Mean squared error (MSE): {:.3f}".format(mean_squared_error(y9_test, y_pred_9)))
print("Modelo 9 - Mean absolute percentage error(MAPE): {:.3f}".format(mean_absolute_percentage_error(y9_test, y_pred_9)))
print("Modelo 9 - R² Score: {:.3f}".format(r2_score(y9_test, y_pred_9)))

#modelo 10 - Random Forest com seleção de variáveis

print("Modelo 10 - Mean absolute error (MAE): {:.3f}".format(mean_absolute_error(y10_test, y_pred_10)))
print("Modelo 10 - Mean squared error (MSE): {:.3f}".format(mean_squared_error(y10_test, y_pred_10)))
print("Modelo 10 - Mean absolute percentage error(MAPE): {:.3f}".format(mean_absolute_percentage_error(y10_test, y_pred_10)))
print("Modelo 10 - R² Score: {:.3f}".format(r2_score(y10_test, y_pred_10)))

#modelo 11 - Random Forest com seleção de variáveis

print("Modelo 11 - Mean absolute error (MAE): {:.3f}".format(mean_absolute_error(y11_test, y_pred_11)))
print("Modelo 11 - Mean squared error (MSE): {:.3f}".format(mean_squared_error(y11_test, y_pred_11)))
print("Modelo 11 - Mean absolute percentage error(MAPE): {:.3f}".format(mean_absolute_percentage_error(y11_test, y_pred_11)))
print("Modelo 11 - R² Score: {:.3f}".format(r2_score(y11_test, y_pred_11)))

#### 1.7.6 Resultados da previsão de receita

O modelo campeão foi o modelo 10 por apresentar o maior coeficiente de correlação. Alem disso, foi treinado com as variáveis 
selecionadas pelo modelo Random Forest Regression.

O modelo 1 não passou por nenhum método de seleção de variáveis e o modelo 11, apesar de ter seus hiperparâmetros otimizados
pelo grid search, apresentou o maior MAPE. Devido a essas informações, os modelos não foram escolhidos. 

As previsões da quantidade bruta de vendas e da receita líquida para os materias do cluster 3 são importantes
para conhecermos o comportamento da venda, além disso, traz benefícios como:

- Controle de estoque (Maior acuracidade e menos custos com estoque excessivo).
- Com a previsão é possível encontrar pontos de maior venda e elaborar ações de marketing eficientes.
- Traz a possibilidade de preparar a equipe de vendas para uma alta demanda.
- Possibilita gerar expectativas corretas sobre o faturamento esperado.
- Ajuda a empresa a conhecer melhor o seu público-alvo e direcionar uma abordagem de vendas mais assertiva.